## Parse generated captions from text file

**DISCLAIMER**: The deep learning model can generate a lot of captions, some of which are unnecessary and overlap with others. I personally deleted those that I didn't want to include (and made sure that the captions I kept didn't overlap for timeperiod). You can follow a different route if you choose. Just be aware that there is some hard-coding that takes place in the MoviePy code section (the part that overlays captions on videos) so you'll need to modify it if you have more than 2 captions.

In [59]:
text_file = open("./sample_automated_captions/skiing_video_output2.txt") #Change to the path of the text file with the generated caption output
text_file_lines = text_file.readlines() #Extract all lines from file
text_file.close()

In [60]:
generated_captions = text_file_lines[-1] #Select last element (last line) to get automated captions
generated_captions = generated_captions.rstrip('\n') #Remove trailing new line characters
generated_captions

"[{'start': 0.2, 'end': 6.5, 'sentence': 'A man is seen standing on a snowy hill holding a stick and holding a stick'}, {'start': 6.5, 'end': 8.0, 'sentence': 'A man in a black shirt is standing in front of a snow'}]"

In [61]:
generated_captions = generated_captions.replace("]", "") #Remove square brackets
generated_captions = generated_captions.replace("[", "")

generated_captions = generated_captions.replace("}, ", "};") #Replace so that we can use ';' as a delimiter to split the string
generated_captions_list = generated_captions.split(";") #Split string by ';' delimiter

In [62]:
generated_captions_list

["{'start': 0.2, 'end': 6.5, 'sentence': 'A man is seen standing on a snowy hill holding a stick and holding a stick'}",
 "{'start': 6.5, 'end': 8.0, 'sentence': 'A man in a black shirt is standing in front of a snow'}"]

In [63]:
english_parsed_captions_list = [] #Will hold strings converted to dictionaries

for item in generated_captions_list:
    english_parsed_captions_list.append(eval(item)) #Append dictionary version of string

In [64]:
english_parsed_captions_list

[{'start': 0.2,
  'end': 6.5,
  'sentence': 'A man is seen standing on a snowy hill holding a stick and holding a stick'},
 {'start': 6.5,
  'end': 8.0,
  'sentence': 'A man in a black shirt is standing in front of a snow'}]

___________________________________________________________________________________________________________________________________

## Begin Arabic Translations

In [65]:
from google_trans_new import google_translator 
  
translator = google_translator() #Import google translate api

In [66]:
import copy
arabic_parsed_captions_list = [] #Will hold Arabic captions

for li in english_parsed_captions_list: #Make a copy of english captions (Copy the individual dictionaries in the list)
    d2 = li.copy()
    arabic_parsed_captions_list.append(d2)

In [67]:
# arabic_parsed_captions_list = english_parsed_captions_list.copy() #Make a copy of english captions

for i in range(len(english_parsed_captions_list)): #Translate the text from english to Arabic
    arabic_parsed_captions_list[i]['sentence'] = translator.translate(english_parsed_captions_list[i]['sentence'], lang_tgt='ar')

In [68]:
arabic_parsed_captions_list

[{'start': 0.2,
  'end': 6.5,
  'sentence': 'شوهد رجل يقف على تل ثلجي يحمل عصا ويمسك بعصا '},
 {'start': 6.5, 'end': 8.0, 'sentence': 'يقف رجل يرتدي قميصًا أسود أمام ثلج '}]

In [69]:
english_parsed_captions_list

[{'start': 0.2,
  'end': 6.5,
  'sentence': 'A man is seen standing on a snowy hill holding a stick and holding a stick'},
 {'start': 6.5,
  'end': 8.0,
  'sentence': 'A man in a black shirt is standing in front of a snow'}]

___________________________________________________________________________________________________________________________________

## Begin MoviePy Editing

In [317]:
from moviepy.editor import *

my_video = VideoFileClip("./sample_videos/skiing_video.mp4", audio=True) #Replace with path to mp4 file you're using

w,h = moviesize = my_video.size
print(moviesize)
print(w,h)

[1920, 1080]
1920 1080


In [318]:
#Code to maintain aspect ratio if original video is in portrait mode
if my_video.rotation == 90:
    my_video = my_video.resize(my_video.size[::-1])
    my_video.rotation = 0

In [319]:
w,h = moviesize = my_video.size #Check to see if aspect ratio is corrected (if original video was in portrait mode)

print(moviesize)
print(w,h)

(1080, 1920)
1080 1920


In [320]:
#Code to mirror Arabic text (needed so that the Arabic captions in video aren't backwards)
import arabic_reshaper
from bidi.algorithm import get_display

def formatArabicSentences(sentences):
   formatedSentences = arabic_reshaper.reshape(sentences)
   return get_display(formatedSentences)

In [323]:
for i in range(len(arabic_parsed_captions_list)): #Translate the text from english to Arabic
    text_to_be_reveresed = arabic_parsed_captions_list[i]['sentence']
    arabic_parsed_captions_list[i]['sentence'] = formatArabicSentences(text_to_be_reveresed)

In [324]:
arabic_parsed_captions_list #Sanity check to see if reversal worked

[{'start': 0.2,
  'end': 6.5,
  'sentence': ' ﺎﺼﻌﺑ ﻚﺴﻤﻳﻭ ﺎﺼﻋ ﻞﻤﺤﻳ ﻲﺠﻠﺛ ﻞﺗ ﻰﻠﻋ ﻒﻘﻳ ﻞﺟﺭ ﺪﻫﻮﺷ'},
 {'start': 6.5, 'end': 8.0, 'sentence': ' ﺞﻠﺛ ﻡﺎﻣﺃ ﺩﻮﺳﺃ ﺎﺼﻴﻤﻗ ﻱﺪﺗﺮﻳ ﻞﺟﺭ ﻒﻘﻳ'}]

___________________________________________________________________________________________________________________________________

### Arabic captioning for first part of clip

**DISCLAIMER**: You'll need to download a compatible Arabic font (ttf file) if Imagemagick doesn't provide one (Run TextClip.list('font') to see if the list contains an Arabic compatible font; if not, download an Arabic ttf font online; I recommend this one https://github.com/rastikerdar/sahel-font/blob/master/dist/Sahel.ttf)

In [325]:
arabic_clip1 = my_video.subclip(0, arabic_parsed_captions_list[0]['end'])

In [326]:
# Generate a text clip. You can customize the font, color, etc.
txt_clip1 = TextClip(arabic_parsed_captions_list[0]['sentence'], fontsize=48, bg_color = 'black', color='white', font='./Fonts/Sahel.ttf')


# Say that you want it to appear at 2.5% width and 60% depth of screen (Modify as needed)
duration = arabic_parsed_captions_list[0]['end'] - 0
txt_clip1 = txt_clip1.set_pos((0.025, 0.6), relative=True).set_duration(duration)

In [327]:
arabic_captioned_clip1 = CompositeVideoClip([arabic_clip1, txt_clip1]) #Overlay text on arabic clip1

### Arabic captioning for second part of clip

In [328]:
arabic_clip2 = my_video.subclip(arabic_parsed_captions_list[1]['start'])

In [329]:
# Generate a text clip. You can customize the font, color, etc.
txt_clip2 = TextClip(arabic_parsed_captions_list[1]['sentence'], fontsize=60, bg_color = 'black', color='white', font='./Fonts/Sahel.ttf')


# Say that you want it to appear at 10% width and 60% depth of screen (Modify as needed)
duration = my_video.duration - arabic_parsed_captions_list[1]['start'] #Duration is from end of video to start of second caption
txt_clip2 = txt_clip2.set_pos((0.1, 0.6), relative=True).set_duration(duration)

In [330]:
arabic_captioned_clip2 = CompositeVideoClip([arabic_clip2, txt_clip2]) #Overlay text on arabic clip2

### Combine Arabic captioning clips

In [331]:
final_arabic_clip = concatenate_videoclips([arabic_captioned_clip1, arabic_captioned_clip2]) #Combine the 2 arabic captioned clips 

#Write final concatenated video to final mp4 file. All parameters after name of final mp4 file are to make sure audio is included in final product
final_arabic_clip.write_videofile("Arabic_captioned_concatenated_skiing_video.mp4", temp_audiofile='temp-audio.m4a', remove_temp=True, codec="libx264", audio_codec="aac") #Change name of mp4 as needed

t:   0%|          | 0/228 [00:00<?, ?it/s, now=None]                

Moviepy - Building video Arabic_captioned_concatenated_skiing_video.mp4.
MoviePy - Writing audio in temp-audio.m4a
MoviePy - Done.
Moviepy - Writing video Arabic_captioned_concatenated_skiing_video.mp4



Moviepy - Done !
Moviepy - video ready Arabic_captioned_concatenated_skiing_video.mp4


___________________________________________________________________________________________________________________________________

### English captioning for first part of clip

In [420]:
english_clip1 = my_video.subclip(0, english_parsed_captions_list[0]['end'])

In [421]:
# Generate a text clip. You can customize the font, color, etc.
# Needed to break text into 2 pieces in order to fit in video
english_sentence1_part1 = ' '.join(english_parsed_captions_list[0]['sentence'].split()[:9])
english_sentence1_part2 = ' '.join(english_parsed_captions_list[0]['sentence'].split()[9:])

txt_clip1_1 = TextClip(english_sentence1_part1, fontsize=55, bg_color = 'black', color='white', font='Arial')
txt_clip1_2 = TextClip(english_sentence1_part2, fontsize=55, bg_color = 'black', color='white', font='Arial')

# Say that you want it to appear at 8% & 11% width and 60% & 63.5% depth of screen (Modify as needed)
duration = english_parsed_captions_list[0]['end'] - 0
txt_clip1_1 = txt_clip1_1.set_pos((0.08, 0.6), relative=True).set_duration(duration)
txt_clip1_2 = txt_clip1_2.set_pos((0.11, 0.635), relative=True).set_duration(duration)

In [422]:
english_captioned_clip1 = CompositeVideoClip([english_clip1, txt_clip1_1, txt_clip1_2]) #Overlay text on english clip1

### English captioning for second part of clip

In [423]:
english_clip2 = my_video.subclip(english_parsed_captions_list[1]['start'])

In [424]:
# Generate a text clip. You can customize the font, color, etc.
txt_clip2 = TextClip(english_parsed_captions_list[1]['sentence'], fontsize=45, bg_color = 'black', color='white', font='Arial')


# Say that you want it to appear at 2.5% width and 60% depth of screen (Modify as needed)
duration = my_video.duration - english_parsed_captions_list[1]['start'] #Duration is from end of video to start of second caption
txt_clip2 = txt_clip2.set_pos((0.025, 0.6), relative=True).set_duration(duration)

In [425]:
english_captioned_clip2 = CompositeVideoClip([english_clip2, txt_clip2]) #Overlay text on english clip2

### Combine English captioning clips

In [426]:
final_english_clip = concatenate_videoclips([english_captioned_clip1, english_captioned_clip2]) #Combine the 2 english captioned clips 

#Write final concatenated video to final mp4 file. All parameters after name of final mp4 file are to make sure audio is included in final product
final_english_clip.write_videofile("English_captioned_concatenated_skiing_video.mp4", temp_audiofile='temp-audio.m4a', remove_temp=True, codec="libx264", audio_codec="aac") #Change name of mp4 as needed

t:   0%|          | 0/228 [00:00<?, ?it/s, now=None]              

Moviepy - Building video English_captioned_concatenated_skiing_video.mp4.
MoviePy - Writing audio in temp-audio.m4a
MoviePy - Done.
Moviepy - Writing video English_captioned_concatenated_skiing_video.mp4



Moviepy - Done !
Moviepy - video ready English_captioned_concatenated_skiing_video.mp4


___________________________________________________________________________________________________________________________________

## Join Arabic and English Captioned videos together

In [427]:
final_concatenated_video = concatenate_videoclips([final_arabic_clip, final_english_clip]) #Combine the Arabic and English captioned clips 

#Write final concatenated video to final mp4 file. All parameters after name of final mp4 file are to make sure audio is included in final product
final_concatenated_video.write_videofile("Final_concatenated_skiing_video.mp4", temp_audiofile='temp-audio.m4a', remove_temp=True, codec="libx264", audio_codec="aac") #Change name of mp4 as needed

chunk:  19%|█▊        | 62/334 [00:00<00:00, 617.92it/s, now=None]

Moviepy - Building video Final_concatenated_skiing_video.mp4.
MoviePy - Writing audio in temp-audio.m4a


t:   1%|          | 3/455 [00:00<00:16, 27.06it/s, now=None]        

MoviePy - Done.
Moviepy - Writing video Final_concatenated_skiing_video.mp4



Moviepy - Done !
Moviepy - video ready Final_concatenated_skiing_video.mp4
